In [31]:
import pandas as pd
import numpy as np
from dateutil import parser

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
sns.set(style="darkgrid", color_codes=True)

from googleapiclient.discovery import build

In [22]:
api_key = '' 

channel_ids = ['UCUTuOCgFBGp2yag_FtVh-bQ',  #simplydaisy	
               'UCQEzmjboJ_6-uG8-1j4coNw', #bellacoco
               'UCeoIctYCZjKJUl2I25hhUvQ', #PlanetJune
               'UCGQC3hnDEFuN7LnOMogLhwQ', #Knitting my hobby
               'UCnd8o8H2-r4mujlVeDVOtaQ', #HappyBerry Crochet
               'UC293v8l6hd3sfe6s-iHM0RA', #forthefrills
               'UCClYzurwbHhwP8Senujmf1A', #Craft & Crochet
               'UCOAJoxOnmQqdPe7AeRTdloA', #brittanyalicecrochet
               'UCClZAyB78QskB8vqC7D_Zbw', #ikoxun
               'UCVbbBdiTKMCx5VII4IKDU7A', #thecrochetcrowd
               'UCNDSOZh_-SJxhosMSPwZ7gQ', #RichTexturesCrochet
               'UCYdtpWueAERC0OnN8RyWDIA', #elenarugal
               'UCbNijc2seCn3ZKNwOOLrnzQ', #LePetitSaintCrochet
               'UCtb80P7saiNRY126C_BGA4Q', #CrochetWithTiffany
               ] 


youtube = build('youtube', 'v3', developerKey=api_key)

In [23]:
def get_channel_stats(youtube, channel_ids):
    
    all_data = []
    
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()

    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
        }
        
        all_data.append(data)
        
    return pd.DataFrame(all_data)

In [24]:
def get_channel_stats(youtube, channel_ids):
    all_data = []
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids))
    response = request.execute() 
    
    for i in range(len(response['items'])):
        data = dict(channelName = response['items'][i]['snippet']['title'],
                    subscribers = response['items'][i]['statistics']['subscriberCount'],
                    views = response['items'][i]['statistics']['viewCount'],
                    totalVideos = response['items'][i]['statistics']['videoCount'],
                    playlistId = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)
    
    return pd.DataFrame(all_data)

def get_video_ids(youtube, playlist_id):
  
    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    video_ids = []
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids

def get_video_details(youtube, video_ids):
        
    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
            
    return pd.DataFrame(all_video_info)

In [25]:
channel_data = get_channel_stats(youtube, channel_ids)
channel_data

,channelName,subscribers,views,totalVideos,playlistId
0,Rich Textures Crochet,195000,15174355,602,UUNDSOZh_-SJxhosMSPwZ7gQ
1,simplydaisy,422000,40311618,29,UUUTuOCgFBGp2yag_FtVh-bQ
2,Crochet With Tiffany Hansen,249000,15288249,483,UUtb80P7saiNRY126C_BGA4Q
3,The Crochet Crowd,1180000,184825614,4170,UUVbbBdiTKMCx5VII4IKDU7A
4,Knitting my hobby,511000,90520887,828,UUGQC3hnDEFuN7LnOMogLhwQ
5,Crochet.ElenaRugalStudio,926000,157704408,1549,UUYdtpWueAERC0OnN8RyWDIA
6,LePetitSaint Crochet,80400,4557082,234,UUbNijc2seCn3ZKNwOOLrnzQ
7,Bella Coco,1310000,149201773,654,UUQEzmjboJ_6-uG8-1j4coNw
8,Wilma Westenberg,221000,18072029,179,UU293v8l6hd3sfe6s-iHM0RA
9,ikoxun,198000,8027919,60,UUClZAyB78QskB8vqC7D_Zbw


In [26]:
channel_data['totalVideos'].astype('int').sum()

9388

In [27]:
video_df = pd.DataFrame()

for c in channel_data['channelName'].unique():
    print("Getting video information from channel: " + c)
    playlist_id = channel_data.loc[channel_data['channelName']== c, 'playlistId'].iloc[0]
    video_ids = get_video_ids(youtube, playlist_id)
    video_data = get_video_details(youtube, video_ids)
    video_df = video_df.append(video_data, ignore_index=True)

Getting video information from channel: Rich Textures Crochet


<ipython-input-27-9ce0009dcc6d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  video_df = video_df.append(video_data, ignore_index=True)


Getting video information from channel: simplydaisy


<ipython-input-27-9ce0009dcc6d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  video_df = video_df.append(video_data, ignore_index=True)


Getting video information from channel: Crochet With Tiffany Hansen


...

<ipython-input-27-9ce0009dcc6d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  video_df = video_df.append(video_data, ignore_index=True)


Getting video information from channel: PlanetJune


<ipython-input-27-9ce0009dcc6d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  video_df = video_df.append(video_data, ignore_index=True)


Getting video information from channel: HappyBerry Crochet


<ipython-input-27-9ce0009dcc6d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  video_df = video_df.append(video_data, ignore_index=True)


In [28]:
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,SrWrposgVis,Rich Textures Crochet,Meadows Stitch | How to Crochet,The Meadows Crochet Stitch pattern is a wonder...,"[crochet, crochet for beginners, crochet blank...",2023-03-19T13:00:29Z,1546,250,None,25,PT5M42S,hd,false
1,SubYN9meZMM,Rich Textures Crochet,Have you tried the Crossed Box Stitch?,The full video tutorial is here: https://youtu...,None,2023-03-18T13:58:01Z,1586,90,None,1,PT35S,hd,false
2,SZEUBKQyKLk,Rich Textures Crochet,Straight Cables for the Make it Your Way Croch...,The full video tutorial is here: https://youtu...,None,2023-03-17T20:38:17Z,2062,140,None,1,PT39S,hd,false
3,dVi0H_u1sWU,Rich Textures Crochet,The Crochet Elizabeth Stitch,The Full Video Tutorial is here: https://youtu...,None,2023-03-16T14:19:05Z,4045,221,None,5,PT26S,hd,false
4,hajfZVJ9xDA,Rich Textures Crochet,Elizabeth Afghan Square Crochet Pattern - Make...,The Elizabeth Afghan Square features a favouri...,"[crochet, crochet for beginners, crochet blank...",2023-03-15T13:00:36Z,1708,193,None,5,PT10M29S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9244,0m0952XQjbA,HappyBerry Crochet,(Crochet) How to crochet a magic circle loop ring,This video shows you how to make a single croc...,"[crochet, magic circle, magic ring, single cro...",2012-05-28T23:21:38Z,107664,996,None,53,PT7M47S,hd,false
9245,FMdj4j8E9U0,HappyBerry Crochet,(Crochet) Crocheting into foundation chain row...,Ep4 of 4 on how to begin one of my shoe patter...,"[pattern, crochet, foundation, shoe, stitches,...",2012-05-23T19:41:59Z,16519,141,None,2,PT6M29S,hd,false
9246,1ehHCDDv2-0,HappyBerry Crochet,(Crochet) US Single Crochet Stitch SC / UK Dou...,Ep3 of 4 on how to begin one of my HappyBerry ...,"[to, US, happyberry, single, how, crochet, Pre...",2012-05-23T17:35:11Z,29065,164,None,4,PT7M7S,hd,false
9247,0hpDtyS3xCA,HappyBerry Crochet,(Crochet) How To - Crochet a Chain Stitch CH,Crochet instruction video on how to chain your...,"[to, uk, free, happyberry, pattern, chain, how...",2012-05-22T22:01:43Z,5148,72,None,4,PT3M28S,hd,false


In [29]:
video_df.to_csv('video_data_top10_channels.csv')